<a href="https://colab.research.google.com/github/djbell01/563-DawsonBell/blob/main/MC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
class Particle():
  def __init__(self,x,y,z,r):
    self.x = x
    self.y = y
    self.z = z
    self.radius = r

class System():
  def __init__(self,N,L,particle_radius,r_cut):
    self.N = N
    self.L = L
    self.r_cut = r_cut

    # Is it necessary to check for particle overlap?
    # Will the Lennard-Jones potential make such a system
    # essentially impossible anyway?
    # If the first configuration is unphysical, does this
    # matter? Could I end up sampling an unphysical system
    # many times at the beginning? Is that okay as long
    # as I eject the equilibration from any trajectory analysis?
    self.particles = []
    for i in range(self.N):
      x = np.random.uniform(0,self.L)
      y = np.random.uniform(0,self.L)
      z = np.random.uniform(0,self.L)
      self.particles.append(Particle(x,y,z,particle_radius))




SyntaxError: incomplete input (<ipython-input-2-9a69085dc3a7>, line 3)

In [39]:
L=6

print(np.random.uniform(0,L))

3.805637772928928
